### Crime Classification

In [1]:
#import lib

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import graphlab as gpl

A newer version of GraphLab Create (v1.9) is available! Your current version is v1.8.5.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
#Load Data, and parse the first column into datetime

train = pd.read_csv('train.csv', parse_dates = ['Dates'])
test = pd.read_csv('test.csv', parse_dates = ['Dates'])

In [3]:
# train data

#days = pd.get_dummies(train.DayOfWeek)
#district = pd.get_dummies(train.PdDistrict)
#hour = train.Dates.dt.hour
#hour = pd.get_dummies(hour)
#year = train.Dates.dt.year
#year = pd.get_dummies(year)

#month = train.Dates.dt.month
#month = pd.get_dummies(month)
#month.columns = [24,25,26,27,28,29,30,31,32,33,34,35]

#date = train.Dates.dt.day
#date.unique()
#date = pd.get_dummies(date)
#date.columns = list(range(41,72))

#intersection

inter = train.Address.apply(lambda x: 1 if '/' in x else 0)

# crime at night or not

crime_time = train.Dates.dt.hour.apply(lambda x: 1 if (x>=22) or (x<6) else 0)

# weekday/weekend crime

crime_day = train.Dates.dt.dayofweek.apply(lambda x: 1 if (x==5) or (x==6) else 0)

In [4]:
#train_data = pd.concat([hour, days, month, district, loc], axis=1)

train_data = pd.concat([train.PdDistrict, inter, crime_time, crime_day, train.X, train.Y], axis=1)
train_data['crime'] = train.Category

In [5]:
train_data.columns = ['PdDistrict', 'inter', 'crime_time', 'crime_day', 'X', 'Y', 'crime']

In [6]:
#for test data
'''
days = pd.get_dummies(test.DayOfWeek)
district = pd.get_dummies(test.PdDistrict)
hour = test.Dates.dt.hour
hour = pd.get_dummies(hour)  
year = test.Dates.dt.year
year = pd.get_dummies(year)
month = test.Dates.dt.month
month = pd.get_dummies(month)
month.columns = [24,25,26,27,28,29,30,31,32,33,34,35]

date = test.Dates.dt.day
date = pd.get_dummies(date)
date.columns = list(range(41,72))
'''
#intersection

inter = test.Address.apply(lambda x: 1 if '/' in x else 0)

# crime at night or not

crime_time = test.Dates.dt.hour.apply(lambda x: 1 if (x>22) and (x<6) else 0)

# weekday/weekend crime

crime_day = test.Dates.dt.dayofweek.apply(lambda x: 1 if (x==5) or (x==6) else 0)

In [7]:
test_data = pd.concat([test.PdDistrict, inter, crime_time, crime_day, test.X, test.Y], axis=1)

test_data.columns = ['PdDistrict', 'inter', 'crime_time', 'crime_day', 'X', 'Y']

In [8]:
features = list(train_data.columns)[:-1]


## GRAPHLAB

In [9]:
train_dato = gpl.SFrame(train_data)
test_dato = gpl.SFrame(test_data)

2016-05-09 15:21:55,047 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1462787513.log


This non-commercial license of GraphLab Create is assigned to hpanwar25@yahoo.com and will expire on December 16, 2016. For commercial licensing options, visit https://dato.com/buy/.


In [ ]:
train_dato.show()

In [10]:
sf_train, sf_test = train_dato.random_split(.6, seed=5)

In [11]:
model = gpl.random_forest_classifier.create(sf_train, target='crime')

Random forest classifier:

--------------------------------------------------------

Number of examples          : 499690

Number of classes           : 39

Number of feature columns   : 6

...

| 8         | 131.261927   | 0.254960          | 3.005089          | 0.251805            | 3.027153            |

| 9         | 148.072039   | 0.255418          | 2.999047          | 0.252185            | 3.020899            |

| 10        | 164.899788   | 0.256231          | 3.003621          | 0.253743            | 3.025594            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [12]:
pred = model.classify(sf_test)

In [13]:
model.evaluate(sf_test)

{'accuracy': 0.2544845628776194,
 'auc': 0.6188126179093082,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 291
 
 Data:
 +--------------------+-----------------+-------+
 |    target_label    | predicted_label | count |
 +--------------------+-----------------+-------+
 |  FAMILY OFFENSES   |  DRUG/NARCOTIC  |   14  |
 |  STOLEN PROPERTY   |  OTHER OFFENSES |  334  |
 |      TRESPASS      |  VEHICLE THEFT  |   21  |
 |      ASSAULT       |  DRUG/NARCOTIC  |  2595 |
 |     KIDNAPPING     |     ASSAULT     |  115  |
 |       FRAUD        |  MISSING PERSON |   31  |
 |  STOLEN PROPERTY   |   NON-CRIMINAL  |   3   |
 | DISORDERLY CONDUCT |     ASSAULT     |   59  |
 |       FRAUD        |     ASSAULT     |  250  |
 |    DRUNKENNESS     |  OTHER OFFENSES |  381  |
 +--------------------+-----------------+-------+
 [291 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print mor